In [1]:
#basic python packages for plotting and array management
import numpy as np
import matplotlib.pyplot as plt
import random

#for data import
import pandas as pd
from __future__ import division
import os
import sys
from importlib import reload
reload(sys)

#Preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

#Classifiers
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV, PassiveAggressiveClassifier, Perceptron, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier

#Ensemble methods and crossvalidation
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier,RandomForestClassifier, VotingClassifier

#PostProcessing
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import time

In [2]:
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
X_train = pd.read_csv("X_train.csv")
X_test   = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")

In [3]:
# Preprocessing Products
col = products['SizeAdviceDescription'].eq('Prenez votre pointure habituelle').astype('float')
col1 = products['SizeAdviceDescription'].eq(products['SizeAdviceDescription'][7]).astype('float')
products['SizeAdviceDescription'] = col+col1

In [4]:
products.head()

,VariantId,GenderLabel,MarketTargetLabel,SeasonLabel,SeasonalityLabel,BrandId,UniverseLabel,TypeBrand,ProductId,ProductType,...,UpperHeight,HeelHeight,PurchasePriceHT,IsNewCollection,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription
0,728257.0,Homme,Classique,Automne/Hiver,Saisonnier,66.0,DÃ©tente,Standard,17267.0,Baskets,...,NaN,0.0,30.50,0.0,Montantes,NaN,NaN,NaN,False,0.0
1,806356.0,Femme,ND,Automne/Hiver,Saisonnier,842.0,ND,Standard,30824.0,Baskets,...,0.0,0.0,43.00,0.0,Montantes,NaN,NaN,NaN,True,1.0
2,768790.0,Femme,ND,Automne/Hiver,Reconduit,988.0,Ville,Standard,62475.0,Bottines et boots,...,6.0,3.0,54.90,0.0,Bout pointu,NaN,NaN,NaN,False,1.0
3,515679.0,Femme,ND,Automne/Hiver,Saisonnier,769.0,Ville,Standard,43983.0,Escarpins,...,0.0,13.0,34.50,0.0,Bout rond,NaN,NaN,NaN,False,1.0
4,1025246.0,Femme,ND,Automne/Hiver,Saisonnier,1244.0,ND,Standard,81493.0,Bottines et boots,...,8.0,4.0,43.76,0.0,Bout rond,NaN,NaN,NaN,False,1.0


In [5]:
index = pd.DataFrame({'index':np.arange(X_train.shape[0])})
X_train['index']=index # Evite les duplicata
y_train['index']=index

In [6]:
X_Product = pd.merge(X_train,products,how='left',on='VariantId')
X_data = pd.merge(X_Product,customers,how='left',on='CustomerId')
data = pd.merge(y_train,X_data,how='left',on="index")
data = data.sort_values(by='index')
data.head()

,OrderNumber_x,LineItem_x,ReturnQuantity,ReturnQuantityBin,index,OrderNumber_y,VariantId,LineItem_y,CustomerId,OrderStatusLabel,...,SubtypeLabel,UpperMaterialLabel,LiningMaterialLabel,OutSoleMaterialLabel,RemovableSole,SizeAdviceDescription,CountryISOCode,BirthDate,Gender,FirstOrderDate
0,73521754,1,0.0,0.0,0,73521754,439729,1,12443972,Expédié,...,NaN,NaN,NaN,NaN,NaN,NaN,FR,1958-03-05 00:00:00,Femme,2011-10-26 12:10:48
1,73521754,2,0.0,0.0,1,73521754,440174,2,12443972,Expédié,...,NaN,NaN,NaN,NaN,NaN,NaN,FR,1958-03-05 00:00:00,Femme,2011-10-26 12:10:48
2,73525226,1,1.0,1.0,2,73525226,494501,1,12443958,Expédié,...,Bout rond,NaN,NaN,NaN,False,1.0,FR,1972-06-14 00:00:00,Femme,2011-10-26 12:11:38
3,73529009,1,1.0,1.0,3,73529009,439590,1,12443946,Expédié,...,NaN,NaN,NaN,NaN,NaN,NaN,FR,1970-05-14 00:00:00,Femme,2011-10-26 12:13:09
4,73529009,2,1.0,1.0,4,73529009,559476,2,12443946,Expédié,...,Bout rond,Cuir,NaN,NaN,False,1.0,FR,1970-05-14 00:00:00,Femme,2011-10-26 12:13:09


In [7]:
data.columns.values

array(['OrderNumber_x', 'LineItem_x', 'ReturnQuantity',
       'ReturnQuantityBin', 'index', 'OrderNumber_y', 'VariantId',
       'LineItem_y', 'CustomerId', 'OrderStatusLabel', 'OrderTypelabel',
       'SeasonLabel_x', 'PayementModeLabel', 'CustomerTypeLabel',
       'IsoCode', 'DeviceTypeLabel', 'PricingTypeLabel', 'TotalLineItems',
       'Quantity', 'UnitPMPEUR', 'OrderCreationDate', 'OrderShipDate',
       'OrderNumCustomer', 'IsOnSale', 'BillingPostalCode', 'GenderLabel',
       'MarketTargetLabel', 'SeasonLabel_y', 'SeasonalityLabel', 'BrandId',
       'UniverseLabel', 'TypeBrand', 'ProductId', 'ProductType',
       'SupplierColor', 'ProductColorId', 'MinSize', 'MaxSize', 'CalfTurn',
       'UpperHeight', 'HeelHeight', 'PurchasePriceHT', 'IsNewCollection',
       'SubtypeLabel', 'UpperMaterialLabel', 'LiningMaterialLabel',
       'OutSoleMaterialLabel', 'RemovableSole', 'SizeAdviceDescription',
       'CountryISOCode', 'BirthDate', 'Gender', 'FirstOrderDate'], dtype=object)

In [8]:
returndata = data[data['ReturnQuantityBin']==1]

## Products

In [9]:
data.ProductType.value_counts()

Baskets                    181070
Sandales et nu-pieds       102918
Bottines et boots           93170
Ballerines                  53163
Escarpins                   48495
Chaussures Ã  lacets        40245
Chaussures de sport         39761
Bottes                      31906
Tongs                       28287
Mocassins                   17830
Sacs Ã  main                 9390
Mules et sabots              9330
Chaussons                    7635
Chaussures Ã  scratch        5752
Espadrilles                  5553
Produits d'entretien         4020
Petite Maroquinerie          1978
Semelles                     1317
Sacs pochettes               1036
Sacs Ã  dos                   891
Sacs homme                    761
Sacs de sport                 601
Lacets                        529
Bagages                       482
Scolaire                      284
Chaussettes et collants       279
Ceintures                     247
Business                      107
Sacs ordinateur                71
Isotherme     

##### Price HT

In [64]:
print('pour les prix entre 0 et 20€')
# Pour toutes les données
price020 = ((data['PurchasePriceHT']>0) & (data['PurchasePriceHT']<20)).sum()/(data.shape[0])
print(price020)

# Pour les données retournées
price020return = ((returndata['PurchasePriceHT']>0) & (returndata['PurchasePriceHT']<20)).sum()/(returndata.shape[0])
print(price020return)

print('\npour les prix entre 20 et 40€')
# Pour toutes les données
price = ((data['PurchasePriceHT']>20) & (data['PurchasePriceHT']<40)).sum()/(data.shape[0])
print(price)

# Pour les données retournées
pricereturn = ((returndata['PurchasePriceHT']>20) & (returndata['PurchasePriceHT']<40)).sum()/(returndata.shape[0])
print(pricereturn)

print('\npour les prix entre 40 et 60€')
# Pour toutes les données
price = ((data['PurchasePriceHT']>40) & (data['PurchasePriceHT']<60)).sum()/(data.shape[0])
print(price)

# Pour les données retournées
pricereturn = ((returndata['PurchasePriceHT']>40) & (returndata['PurchasePriceHT']<60)).sum()/(returndata.shape[0])
print(pricereturn)

print('\npour les prix entre 60 et 80€')
# Pour toutes les données
price = ((data['PurchasePriceHT']>60) & (data['PurchasePriceHT']<80)).sum()/(data.shape[0])
print(price)

# Pour les données retournées
pricereturn = ((returndata['PurchasePriceHT']>60) & (returndata['PurchasePriceHT']<80)).sum()/(returndata.shape[0])
print(pricereturn)

pour les prix entre 0 et 20€
0.155964170938
0.128892858432

pour les prix entre 20 et 40€
0.295360211376
0.30245660568

pour les prix entre 40 et 60€
0.133006961557
0.152421421096

pour les prix entre 60 et 80€
0.0341912694769
0.0433988307892


##### SeasonLabel

In [17]:
# Pour toutes les données
d = ((data['SeasonLabel_x'] == 'Automne/Hiver').sum())/(data.shape[0])
print(d)

# Pour les données retournées
dreturn = ((returndata['SeasonLabel_x'] == 'Automne/Hiver').sum())/(returndata.shape[0])
print(dreturn)

0.444673893693
0.455599761827


##### GenderLabel

In [19]:
print('femme')
# Pour toutes les données
d = ((data['GenderLabel'] == 'Femme').sum())/(data.shape[0])
print(d)

# Pour les données retournées
dreturn = ((returndata['GenderLabel'] == 'Femme').sum())/(returndata.shape[0])
print(dreturn)

print('\nhomme')
# Pour toutes les données
d = ((data['GenderLabel'] == 'Homme').sum())/(data.shape[0])
print(d)

# Pour les données retournées
dreturn = ((returndata['GenderLabel'] == 'Homme').sum())/(returndata.shape[0])
print(dreturn)

print('\nenfant')
# Pour toutes les données
d = ((data['GenderLabel'] == 'Enfant').sum())/(data.shape[0])
print(d)

# Pour les données retournées
dreturn = ((returndata['GenderLabel'] == 'Enfant').sum())/(returndata.shape[0])
print(dreturn)

femme
0.350230958783
0.427812962362

homme
0.144029270395
0.111476489481

enfant
0.131143363097
0.112486918552


###### Size advice description

In [15]:
# Pour toutes les données
d = ((data['SizeAdviceDescription'] == 1).sum())/(data.shape[0])
print(d)

# Pour les données retournées
dreturn = ((returndata['SizeAdviceDescription'] == 1).sum())/(returndata.shape[0])
print(dreturn)

0.40445989375
0.406142867453


# Customers

##### Gender

In [16]:
print('homme')
# Pour toutes les données
d = ((data['Gender'] == 'Homme').sum())/(data.shape[0])
print(d)

# Pour les données retournées
dreturn = ((returndata['Gender'] == 'Homme').sum())/(returndata.shape[0])
print(dreturn)

print('\nfemme')
# Pour toutes les données
d = ((data['Gender'] == 'Femme').sum())/(data.shape[0])
print(d)

# Pour les données retournées
dreturn = ((returndata['Gender'] == 'Femme').sum())/(returndata.shape[0])
print(dreturn)

homme
0.135259395291
0.102914907437

femme
0.605318142211
0.635090758183


# Orders

##### Customer type label

In [75]:
print('fidélisé')
# Pour toutes les données
d = ((data['CustomerTypeLabel'] == 'Fidélisé').sum())/(data.shape[0])
print(d)

# Pour les données retournées
dreturn = ((returndata['CustomerTypeLabel'] == 'Fidélisé').sum())/(returndata.shape[0])
print(dreturn)

print('\nnouveau')
# Pour toutes les données
d = ((data['CustomerTypeLabel'] == 'Nouveau').sum())/(data.shape[0])
print(d)

# Pour les données retournées
dreturn = ((returndata['CustomerTypeLabel'] == 'Nouveau').sum())/(returndata.shape[0])
print(dreturn)

fidélisé
0.576905995559
0.616560210747

nouveau
0.423094004441
0.383439789253


In [10]:
print('femmes fidélisées')
# Pour toutes les données
price = ((data['CustomerTypeLabel'] == 'Fidélisé') & (data['Gender'] == 'Femme')).sum()/(data.shape[0])
print(price)
#price = ((data['CustomerTypeLabel'] == 'Nouveau') & (data['Gender'] == 'Femme')).sum()/(data.shape[0])
#print(price)

# Pour les données retournées
pricereturn = ((returndata['CustomerTypeLabel'] == 'Fidélisé') & (returndata['Gender'] == 'Femme')).sum()/(returndata.shape[0])
print(pricereturn)

femmes fidélisées
0.38333255254
0.423211901411


In [73]:
print('hommes non fidélisées')
# Pour toutes les données
price = ((data['CustomerTypeLabel'] == 'Nouveau') & (data['Gender'] == 'Homme')).sum()/(data.shape[0])
print(price)

# Pour les données retournées
pricereturn = ((returndata['CustomerTypeLabel'] == 'Nouveau') & (returndata['Gender'] == 'Homme')).sum()/(returndata.shape[0])
print(pricereturn)

hommes non fidélisées
0.0722802612223
0.0513153621306


In [22]:
pd.crosstab(data['Gender'],data['CustomerTypeLabel'],normalize='all')

CustomerTypeLabel,Fidélisé,Nouveau
Gender,,
Femme,0.517612,0.299746
Homme,0.085040,0.097600
NC,0.000003,0.000000


In [23]:
pd.crosstab(returndata['Gender'],returndata['CustomerTypeLabel'],normalize='all')

CustomerTypeLabel,Fidélisé,Nouveau
Gender,,
Femme,0.573450,0.287095
Homme,0.069917,0.069532
NC,0.000006,0.000000
